#### <font color="asparagus"> $\Rightarrow$ Lending Club EDA: <br> <font>
$\Rightarrow$ The objective here is to find the key features/indicators that will help lenders make an informed decision on lending money to a borrower.<br>
$\Rightarrow$ The column "loan_status" has three entries: *Current*, *charged Off* and *Fully Paid*. <br> $\Rightarrow$ People may discard currently running loans and just look at *charged Off* and *Fully Paid* loans for their analysis. But we won't do that.<br> $\Rightarrow$ Rather we will try to find some patterns in currently running loans as well. The entire analysis can be split into the following sections:

### <font color='cyan'> Sections in this notebook: <font>
I. Prerequisites 
    
    I.1. Importing modules, load data
    I.2. Cache necessary information about the data up front

II. Data understanding and cleaning

    II.1. Addressing missing data
        II.1.1. Let's get a feel for the missing data in the df first
        II.1.2. Special cases
    II.2. Format Conversion for certain columns
    II.3. Quasi-Constant Variables
        II.3.1. Numeric
        II.3.2. For all other columns
        II.3.3. Special Cases
    II.4. Object type columns
    II.5. Deriving new columns 
    II.6. Data imputation
    II.7. Correlation
    II.8. Outlier Detection
        II.8.1 Box Plots
        II.8.2. IQR Analysis
    
III. Data Analysis:

    III.1. Univariate Analysis
        III.1.1. Box Plots contd.
        III.1.2. Probability Distribution 
    III.2. Bivariate analysis
        III.2.1. Categorical columns with a keen focus on loan_status
            III.2.1.1. Grade
            III.2.1.2. Home Ownership
            III.2.1.3. Verification Status
            III.2.1.4. Purpose
            III.2.1.5. Employee Experience
            III.2.1.6. Annual Income
            III.2.1.7. Funded Amount
            III.2.1.8. Rate of Interest
            III.2.1.9. Public records
            III.2.1.10. Number of inquiries in the last 6 months
            III.2.1.11. State Address
            III.2.1.12. Loan Term
            III.2.1.13. Delta_bins
        III.2.2. In between rest of the columns(minus loan_status)
    
IV. Conclusion
    
V. Recommendations

# <font color='goldenrod'> I. Prerequisites </font>

### <font color='skyblue'>  I.1. Importing modules, load data<font>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.feature_selection import VarianceThreshold

##### *1.1. Optional settings used for debugging.*

In [ ]:
#pd.set_option('display.max_columns', 20)
#pd.set_option('display.max_rows', 200)
#pd.set_option('display.min_rows', 100)
#pd.set_option('display.expand_frame_repr', True)
pd.get_option("display.max_rows")

##### *1.2. Load data*

In [ ]:
df = pd.read_csv("loan.csv", dtype={"next_pymnt_d": "string"}) # Explicitly specificying dtype for
                                                               # next_pymnt_d column, just to avoid pd warning

### <font color='skyblue'>  I.2. Cache necessary information about the data up front

 <font color="asparagus"> Cache the following info: 
 1. df dimension, will be used for different percentage based calculations
 2. df itself in a separate variable

In [ ]:
df_loan = pd.read_csv("loan.csv", dtype={"next_pymnt_d": "string"})  # Caching original df, may be need it later for quick comparison

In [ ]:
nDfShape = df.shape
nNoOfRows = nDfShape[0]
nNoOfCols = nDfShape[1]

# <font color='goldenrod'> II. Data understanding and cleaning </font>

### <font color='skyblue'>  II.1. Addressing missing data

#### II.1.1.  Let's get a feel for the missing data in the df first <br>
$\Rightarrow$ Instead of manually tackling each column, we will try to automate as much as possible

In [ ]:
df.info()

$\Rightarrow$ <font color="asparagus"> The following function returns the #NaNs and #unique values in one column"

In [ ]:
def get_columnInfo(df, column):
    return [df[column].isna().sum(), df[column].nunique()]

$\Rightarrow$ <font color="asparagus"> Store this information in a dictionary

In [ ]:
columnInfo = {}
for col in df.columns:
    columnInfo[col] = get_columnInfo(df,col)

$\Rightarrow$ <font color="asparagus"> If a column has 60% missing values, we can drop it. This number can be smaller, but for this dataset 60% works

In [ ]:
nNanThreshold = 0.6 

In [ ]:
droppedCols = []
for col in df.columns:
    if (columnInfo[col][0]/nNoOfRows) >= nNanThreshold:
        print("Column ", col, end=" ")
        print("missing: ", round(100*columnInfo[col][0]/nNoOfRows,2), "%")
        droppedCols.append(col)

#### <font color="asparagus"> *Observations and Comments*:<font> 
$\Rightarrow$ From the stdout above, **2** columns have 64.66% and 92.99% missing, while the remaining **54** have 100% missing data.<br> (We can just use dropna() but I like this approach, we can drop things above a threshold at the same time.) <br>
$\Rightarrow$ I have cached this in droppedCols. Let's further analyze and drop all these columns at once at the end right before k-variate analysis

#### II.1.2. Special cases: id, member_id and url are not features! <br>
#### <font color="asparagus"> *Each entry will be unique, whose description from the data dictionary supplied to us is as follows:*<font>
- member_id: A unique LC assigned Id for the borrower member.
- id: A unique LC assigned ID for the loan listing.
- url: URL for the LC page with listing data.<br>
<font color="asparagus"> $\Rightarrow$ *Side note*:<font>
    
- Even if all of *id* was not unique and there was repetition,  keeping *url* will still not make sense, since it only contains id information. 
Eg: For id: 1077501, the url is https://lendingclub.com/browse/loanDetail.action?loan_id=1077501. Really doesn't make sense to keep url.
- Add these three columns to the list of columns to be dropped
    

In [ ]:
df.id.nunique(), df.member_id.nunique(), df.url.nunique() # Proving that the values are unique : ) 

In [ ]:
droppedCols.append("id")
droppedCols.append("member_id")
droppedCols.append("url")

### <font color='skyblue'>  II.2. Format Conversion for certain columns<font>
- term
- int_rate
- emp_length
- revol_util

##### <font color="asparagus"> $\Rightarrow$ 1. *term*:<font>

In [ ]:
df.term.describe(), df.term.unique()

##### <font color="asparagus"> $\Rightarrow$ *Makes sense removing the string " months" and storing it as an int*<font>

In [ ]:
df.term = df.term.str.replace(" months", "")
df.term = df.term.astype(int)

In [ ]:
df.term.unique()

##### <font color="asparagus"> $\Rightarrow$ *Converting months to years* <font>

In [ ]:
df.term = df.term//12

In [ ]:
df.term.unique()

##### <font color="asparagus"> $\Rightarrow$ 2. *int_rate*:<font> 

In [ ]:
df.int_rate.describe()

##### <font color="asparagus"> $\Rightarrow$ *We can get rid of the %* <font>

In [ ]:
df.int_rate = df.int_rate.str.replace("%","")

In [ ]:
df.int_rate = df.int_rate.astype(float)

##### <font color="asparagus"> $\Rightarrow$ 3. *emp_length* <font>

In [ ]:
df.emp_length.unique()

##### <font color="asparagus"> $\Rightarrow$ *We can get rid of "year" & "years"* <font>

In [ ]:
df.emp_length = df.emp_length.str.replace(" years","")
df.emp_length = df.emp_length.str.replace(" year", "")
df.emp_length = df.emp_length.str.replace("+","")
df.emp_length = df.emp_length.str.replace("< ","")
df.emp_length = df.emp_length.str.replace("10","11")

In [ ]:
df.emp_length.value_counts()

#### <font color="asparagus"> *Comments*:<font> 
$\Rightarrow$ I think it will be more meaningful to categorize this column into bins: Something like: 0-2, 2-4, 4-6, 6-8, 8-10 and >10

$\Rightarrow$ But before that let's handle the null values first. I'm just gonna fill it up with the mode value, since missing values % = 1075/39717, isn't much. 

In [ ]:
df.emp_length.isna().sum()

In [ ]:
df.emp_length = df.emp_length.fillna(df.emp_length.mode()[0])

In [ ]:
df.emp_length = df.emp_length.astype(int)

In [ ]:
emp_length_categories_list = [0, 2, 4, 6, 8, 10, 12]
emp_length_labels_list = ["0-2", "2-4", "4-6", "6-8", "8-10", "> 10"]
df["emp_length_bins"] = pd.cut(df.emp_length, bins=emp_length_categories_list,
                              labels=emp_length_labels_list)

In [ ]:
df.emp_length_bins.value_counts()

##### <font color="asparagus"> $\Rightarrow$ 3. *revol_util* <font>

In [ ]:
df.revol_util.describe()

In [ ]:
df.revol_util = df.revol_util.str.replace("%","") # Getting rid of "%"
df.revol_util= df.revol_util.astype("float")

### <font color='skyblue'> II.3. Quasi-Constant Variables
* Inspired by [Towards Data Science blog on this topic](https://towardsdatascience.com/how-to-detect-constant-quasi-constant-features-in-your-dataset-a1ab7aea34b4)
* Check out [sklearn.feature_selection.VarianceThreshold](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.VarianceThreshold.html) for more info!

####  II.3.1. Numeric data <font>
#### <font color="asparagus"> $\Rightarrow$ Let's drop quasi-constant features where 95% of the values are similar or constant on all the numeric columns <font>

In [ ]:
df_numeric = df.select_dtypes(include=np.number)

In [ ]:
df_numeric.shape[1]

In [ ]:
sel = VarianceThreshold(threshold=0.05)

sel.fit(df_numeric.iloc[:,:])


#### <font color="asparagus"> *Comments*:<font>
$\Rightarrow$ Once it is fit, the support of the vector, sel will give me all the retained features.<br>
$\Rightarrow$ Therefore anything that does not appear in df.columns[support()] are *quasi-constant*
    

In [ ]:
quasi_constant_features_list = [x for x in df_numeric.columns if x not in df_numeric.columns[sel.get_support()]]


In [ ]:
len(quasi_constant_features_list), quasi_constant_features_list

#### <font color="asparagus"> $\Rightarrow$ Remember, we already have identified many columns to be dropped, so there might be an overlap. So let's check the newly identified columns

In [ ]:
newly_identified_cols = [col for col in quasi_constant_features_list if col not in droppedCols]

#### <font color="asparagus"> $\Rightarrow$ The names of the newly identified columns are:

In [ ]:
newly_identified_cols

#### <font color="asparagus"> $\Rightarrow$ If you look at the unique values of these newly identified columns, you would find that besides pub_rec_bankruptcies, the others are straight up constants and not just quasi-constant.<br><font><br>$\Rightarrow$ But I think we should not drop pub_rec_bankruptcies altogther. This seems like an important variable to analyze against loan status(we will do this in bivariate analysis)
#### <font color="asparagus"> $\Rightarrow$ Either way, it makes sense to drop these columns too

In [ ]:
newly_identified_cols.pop(-2) # popping pub_rec_bankruptcies at index -2

In [ ]:
newly_identified_cols

In [ ]:
[df[col].nunique() for col in newly_identified_cols]
    

In [ ]:
[droppedCols.append(col) for col in newly_identified_cols]

#### II.3.2. For all other columns

#### <font color="asparagus"> $\Rightarrow$ columnInfo already has the number of unique values for each column, we had cached it earlier.<font>
#### <font color="asparagus"> $\Rightarrow$ Now is the time to add any column that has only **one** unique to the list of columns to be dropped : ) <font>

In [ ]:
# [droppedCols.append(col) if (df[col].nunique() == 1 & col not in droppedCols) for col in columnInfo] List comprehension to do this..
print("The following columns have only 1 value throughout the data")
for col in columnInfo:
    if (df[col].nunique() == 1) & (col not in droppedCols):
        print(f"col: {col} ")
        droppedCols.append(col)

#### II.3.3. Special Cases
#### <font color="asparagus"> $\Rightarrow$ It's important to note that *desc* column is pointless to have. It contains sentences and words that the borrower filled out when asking for a loan. Now we already have a column called *purpose* which as the name implies holds the purpose for which the loan was taken.<font>
#### <font color="asparagus"> $\Rightarrow$It has 7 unique values, so we will stick with *purpose* column and drop *desc* altogether, unless we want to do sentiment analysis : )<font>

In [ ]:
df.desc.unique() # Just an example of the contents of desc column

In [ ]:
droppedCols.append("desc")

#### <font color="asparagus"> *Comments*: <br> $\Rightarrow$ Finally, let's drop all the columns we have accumulated so far <font>

In [ ]:
nNoOfCols = df.shape[1]
print(nNoOfCols)

In [ ]:
print(f"Number of columns to be dropped {len(set(droppedCols))}, Remainder: {nNoOfCols - len(set(droppedCols))}")

In [ ]:
df.drop(droppedCols,axis=1, inplace=True)

In [ ]:
nNoOfCols = df.shape[1] # Updating nNoOfCols

In [ ]:
nNoOfCols

### <font color='skyblue'>  II.4. Object type columns <font>

#### <font color="asparagus"> *I generally do not like to leave object type columns as is. There could be some inconsistencies in the way data is filled.*<font> 
#### <font color="asparagus"> $\Rightarrow$ Let's sanitize them and explicitly convert them to appropriate formats <font>

In [ ]:
df_object = df.select_dtypes(include="object")

#### <font color="asparagus"> Just checking what kind of values they take

In [ ]:
for col in df_object.columns:
    print("col :", col, end=" ")
    print(columnInfo[col])

#### <font color="asparagus"> Don't want to flood stdout, just printing the unique values for columns with less than 16 unique values <font>

In [ ]:
for col in df_object.columns:
    if columnInfo[col][1] <= 15:
        print("col :", col, end=" ")
        print(df[col].unique())

#### <font color="asparagus"> All of these columns can first be converted to string type, since none of them have any other problems <font>

In [ ]:
for col in df_object.columns:
    df[col] = df[col].astype("string")

### <font color='skyblue'>  II.5. Deriving new columns
    

##### <font color="asparagus"> We've already converted term in months to years. Now let's look at other columns. Starting with: <font>
##### <font color="asparagus"> $\Rightarrow$ 1. *annual_inc*:<font>
<font color="asparagus"> I think we can get a good analysis if we successfully categorize this column into bins of 20000 dollars. 

In [ ]:
df.annual_inc.describe()

##### <font color="asparagus"> $\Rightarrow$ Just checking few things about the annual_inc column <font>

In [ ]:
np.quantile(df.annual_inc,0.99) ## 99% of the borrowers have annual inc < $234,000

In [ ]:
len(df[(df.annual_inc > np.quantile(df.annual_inc,0.99))])

##### <font color="asparagus"> $\Rightarrow$ There are still 43 borrowers(out of 398) whose income is in the top 1%, yet they defaulted. > 10%

In [ ]:
len(df[(df.annual_inc > np.quantile(df.annual_inc,0.99)) & (df.loan_status == "Charged Off")])

In [ ]:
annual_inc_categories_list = [0, 20000, 40000, 60000, 80000, 100000, 6000000]
annual_inc_labels_list = ["0-20", "20-40", "40-60", "60-80", "80-100", "> 100"]
df["annual_inc_bins"] = pd.cut(df.annual_inc, bins=annual_inc_categories_list,
                              labels=annual_inc_labels_list)

In [ ]:
df.annual_inc_bins.value_counts()

##### <font color="asparagus"> $\Rightarrow$ 2. *funded_amnt*:<font>
<font color="asparagus"> I think we can get a good analysis if we successfully categorize this column as well.(bins of 5,000)

In [ ]:
df.funded_amnt.describe()

In [ ]:
funded_amnt_categories_list = [0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000]
funded_amnt_labels_list = ["0-5", "5-10", "10-15", "15-20", "20-25", "25-30", "30-35", "35-40"]
df['funded_amnt_bins'] = pd.cut(df.funded_amnt, bins=funded_amnt_categories_list,
                              labels=funded_amnt_labels_list)

In [ ]:
df.funded_amnt_bins.value_counts()

##### <font color="asparagus"> $\Rightarrow$ 3. *int_rate*:<font>
<font color="asparagus"> I think we can get a good analysis if we successfully categorize this column as well.

In [ ]:
int_rate_range_list = [0, 8, 10, 12, 14, 16, 30]
int_rate_labels_list = ["0-8", "8-10", "10-12", "12-14", "14-16", ">16"]
df["int_rate_bins"] = pd.cut(df["int_rate"], bins=int_rate_range_list, labels=int_rate_labels_list)

In [ ]:
df["int_rate_bins"].value_counts()

##### <font color="asparagus"> $\Rightarrow$ 4. *issue_year and last_payment_year*:<font>
<font color="asparagus"> I think if we extract the issue year and last_payment_year, we can do some sort of analysis 

In [ ]:
mapper ={"Jan": 1, "Feb" : 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6, 
         "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}

In [ ]:
df["issue_year"] = df.issue_d.str.extract(r"\w+\-(\d+)", expand=True)
df["issue_month"] = df.issue_d.str.extract(r"(\w+)\-\d+", expand=True)
df["issue_month"] = df["issue_month"].apply(lambda x: mapper[x])
df["issue_month"] = df["issue_month"].astype(float)
df["issue_year"] = df["issue_year"].astype(float)

In [ ]:
df["issue_year"] = round(df["issue_year"] + df["issue_month"]/12,2)

##### <font color="asparagus"> $\Rightarrow$ last_payment_year <font> 
last_payment_year and last_payment_month first. They both have 71 missing entries

In [ ]:
df["last_payment_year"] = df.last_pymnt_d.str.extract(r"\w+\-(\d+)", expand=True)
df["last_payment_month"] = df.last_pymnt_d.str.extract(r"(\w+)\-\d+", expand=True)

In [ ]:
df["last_payment_year"].fillna(df.last_payment_year.mode()[0], inplace=True)

##### <font color="asparagus"> $\Rightarrow$ Fill last_payment_month with the most common entry for the year 2013(the most common year and the one that we just filled the missing values in last_payment_year with <font> 

In [ ]:
df[df.last_payment_year == df.last_payment_year.mode()[0]]["last_payment_month"].value_counts()

In [ ]:
df["last_payment_year"] = df.last_pymnt_d.str.extract(r"\w+\-(\d+)", expand=True)
df["last_payment_year"].fillna(df.last_payment_year.mode()[0], inplace=True)
df["last_payment_month"] = df.last_pymnt_d.str.extract(r"(\w+)\-\d+", expand=True)
df["last_payment_month"] = df["last_payment_month"].fillna("Mar")


df["last_payment_month"] =df["last_payment_month"].apply(lambda x: mapper[x])
df["last_payment_month"] = df["last_payment_month"].astype(float)
df["last_payment_year"] = df["last_payment_year"].astype(float)

In [ ]:
df["last_payment_year"] = round(df["last_payment_year"] + df["last_payment_month"]/12,2)

In [ ]:
df["delta"] = round(df["last_payment_year"] - df["issue_year"],2)

In [ ]:
delta_list = [0, 1, 2, 3, 4, 5, 6, 7, 8]
delta_labels_list = ["0-1", "1-2", "2-3", "3-4", "4-5", "5-6", "6-7", "7-8"]
df["delta_bins"] = pd.cut(df.delta, bins=delta_list,
                              labels=delta_labels_list)

In [ ]:
df["delta_bins"].value_counts()

### <font color='skyblue'>  II.6. Data Imputation for missing rows

In [ ]:
df.isna().sum()

#### <font color="asparagus"> Low hanging fruits: <font>
1. title 
2. revol_util
3. last_pymnt_d
4. last_credit_pull_d
5. emp_length
6. pub_rec_bankruptcies

##### <font color="asparagus"> $\Rightarrow$ 1. *title*<font> 
- Only 11/39717 have missing values, we can impute using value of mode 

In [ ]:
df.title.fillna(df.title.mode()[0], inplace=True)

##### <font color="asparagus"> $\Rightarrow$ 2. *revol_util*<font> 
- Only 50/39717 have missing values, we can impute using value of mode 

In [ ]:
df.revol_util.fillna(df.revol_util.mode()[0], inplace=True)

##### <font color="asparagus"> $\Rightarrow$ 3. *last_pymnt_d*<font> 
- Only 71/39717 have missing values, we can impute using value of mode 

In [ ]:
df.last_pymnt_d.fillna(df.last_pymnt_d.mode()[0], inplace=True)

##### <font color="asparagus"> $\Rightarrow$ 4. *last_credit_pull_d*<font> 
- Only 2/39717 have missing values, we can impute using value of mode 

In [ ]:
df.last_credit_pull_d.fillna(df.last_credit_pull_d.mode()[0], inplace=True)

In [ ]:
df.isna().sum()

##### <font color="asparagus"> $\Rightarrow$ 5. *emp_length* <font>
- Though it has more missing values than the above ones we just discussed 1075/39717, isn't much. This column is categorical, plus it seems the company has more 10+ experience borrowers. We can fill with mode

##### <font color="asparagus"> $\Rightarrow$ 6. *pub_rec_bankruptcies* <font>

In [ ]:
df["pub_rec_bankruptcies"].value_counts()

In [ ]:
df["pub_rec_bankruptcies"].fillna(df["pub_rec_bankruptcies"].mode()[0], inplace=True)

In [ ]:
df["pub_rec_bankruptcies"].isna().sum()

### <font color='skyblue'>  II.7. Correlation<font>

#### <font color="asparagus"> *Need to create a heatmap of all the retained numeric variables and check the relationship* <font>

In [ ]:
# Define the heatmap parameters
pd.options.display.float_format = "{:,.2f}".format

In [ ]:
df_numeric = df.select_dtypes(include=np.number)

In [ ]:
corr_mat = df_numeric.corr()

# Mask the upper part of the heatmap
mask = np.triu(np.ones_like(corr_mat, dtype=bool))

# Choose the color map
cmap = "viridis"

corr_mat[(corr_mat < 0.1) & (corr_mat > -0.1)] = 0 # Easier to view, don't really care obout weak correlations
# plot the heatmap
plt.figure(figsize=(30,30))
sns.heatmap(corr_mat, mask=mask, vmax=1.0, vmin=-1.0, linewidths=0.1,
            annot_kws={"size": 8, "color": "black"}, square=True, cmap=cmap, annot=True)
plt.show()
plt.tight_layout()

#### <font color="asparagus"> 1. We need to make a decision on these highly correlation variables.<font> 
#### <font color="asparagus"> 2. No need to keep them all, rather, we can discard all but one and continue <font>

##### Strengh of relationship

* |r| < 0.3 $\Rightarrow$ None or Very Weak
* 0.3 < |r| < 0.5 $\Rightarrow$ Weak
* 0.5 < |r| < 0.7 $\Rightarrow$ Moderate
* |r| > 0.7 $\Rightarrow$ Strong

In [ ]:
colsToDrop_fromCorr = [] # New list to maintain the columns we want to drop

#### <font color="asparagus"> $\Rightarrow$ *Focusing on funded_amount, loan_amount, funded_amount_inv first* <font>
1. loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
2. funded_amnt: The total amount committed to that loan at that point in time.
3. funded_amnt_inv: The total amount committed by investors for that loan at that point in time.<br>
The correlation factor, r, is almost 1 for these guys
    
In short, a borrower approaches the company to ask for an amount, *loan_amnt*. The company agrees for a certain amount, *funded_amnt*. *funded_amnt_inv*: Sometimes, the lending club themselves step in and fund the amount. This is that. <br>
$\Rightarrow$ Conclusion: We can just keep *funded_amnt* and discard the rest.We could have easily done our analysis with loan_amnt and dropped funded_amnt instead and our observations wouldn't have changed, because *loan_amnt* will satisfy the condition:<br>
    *funded_amnt* <= *loan_amnt* 

    

In [ ]:
colsToDrop_fromCorr.append("loan_amnt")
colsToDrop_fromCorr.append("funded_amnt_inv")

#### <font color="asparagus"> $\Rightarrow$ *Commenting on installment* <font>

#### *installment* is the monthly payment owed by the borrower if the loan originates as per the data dictionary. <br>
#### Again, r >0.9 with "loan_amnt", "funded_amnt", "funded_amnt_inv". <br>
#### Therefore, *installment* can be dropped as well.

In [ ]:
colsToDrop_fromCorr.append("installment")

#### <font color="asparagus"> $\Rightarrow$ *Commenting on total_xx columns* 
This is information that is not visible at the time of loan application, because many fields like *total_pymnt* will fill up with time once loan is approved. But we will still analyze them.  <font>

#### <font color="asparagus"> 1. total_pymnt: Payments received to date for total amount funded<font>
#### <font color="asparagus"> 2. total_pymnt_inv: Payments received to date for portion of total amount funded by investors<font>
#### <font color="asparagus"> 3. total_rec_prncp: Principal received to date<font>
#### <font color="asparagus"> 4. total_rec_int: Interest received to date<font> 
##### <font color="asparagus"> $\Rightarrow$ We cannot drop total_pymnt, but as it stands, it does not give us good information about the borrower's current status of payment(how much left, how much paid). They are just raw numbers. <br>
##### <font color="asparagus"> $\Rightarrow$ What we can do instead is add a new column: Percentage of funded amount paid.
If a borrower pays the entire amount, the percentage will 100%. Formula: <br>(total_pymnt - total_rec_int - total_rec_late_fee)/funded_amnt.<br> 
#### *Side note*: These columns also show a high correlation with *"loan_amnt", "funded_amnt", "funded_amnt_inv"* (~0.75)<br>

**Conclusion**: We can hold on to *total_pymnt* and discard remaining. By the way, *funded_amnt* is highly correlated with *total_pymnt* as well
    

In [ ]:
df["Pct_Payment_Received"] = round(100 * (df.total_pymnt - df.total_rec_int - df.total_rec_late_fee)/df.funded_amnt,2) 

In [ ]:
colsToDrop_fromCorr.append("total_pymnt_inv")
colsToDrop_fromCorr.append("total_rec_prncp")
colsToDrop_fromCorr.append("total_rec_int")

In [ ]:
colsToDrop_fromCorr


#### <font color="asparagus"> $\Rightarrow$ *Commenting on (total_acc, open_acc): r = 0.69 and (collection_recovery_fee, recoveries)*: r = 0.8 <font>

#### 1. total_acc: The total number of credit lines currently in the borrower's credit file
#### 2. open_acc: The number of open credit lines in the borrower's credit file.

$\Rightarrow$ Makes sense that they are highly correlated. I think we can just **keep the number of open credit lines** and **drop the total**, because **total will include open and closed**. **We are only interested in the ones that are currently running**(this is where we can find whether the borrower is about to default or not)

In [ ]:
colsToDrop_fromCorr.append("total_acc")

#### 1. recoveries: post charge off gross recovery
#### 2. collection_recovery_fee: post charge off collection fee<br>
$\Rightarrow$ Both of these quantities are concerned with post charge off scenarios. Most of the values are 0 for both of them as well. I think it is safe to drop recoveries

In [ ]:
colsToDrop_fromCorr.append("recoveries")

In [ ]:
colsToDrop_fromCorr

In [ ]:
df.drop(colsToDrop_fromCorr,axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
nNoOfCols = df.shape[1]

In [ ]:
nNoOfCols

### <font color='skyblue'>  II.8. Outlier Detection<font>
1. Box Plots<br>
2. Mahalanobis Distance

When describing relationship between two variables, correlations is necessary, but not sufficient. More plots will help

#### <font color="asparagus"> *Sequence*:<font>
$\Rightarrow$ <font color="asparagus"> Boxplot first then  
$\Rightarrow$ <font color="asparagus">  I'm gonnna calculate the IQRs for every column and store the locations of every row that lies outside the 1.5IQR range

### <font color='skyblue'>  II.8.1 Box Plots<font>

In [ ]:
df_numeric = df.select_dtypes(include=np.number)

In [ ]:
#fig, axes = plt.subplots(7,2,figsize=(20,30))
for col in df_numeric.columns:
    plt.figure(figsize=(10,6))
    sns.boxplot(x="loan_status", y=col,
                data=df)
    plt.show()
    

#### <font color="asparagus"> *Observations and Comments*:<font> 
$\Rightarrow$ <font color="asparagus"> Looking at the box plot, it looks like the median(and range) of percentage_payment_received is the lowest for "Charged Off" borrowers. This is a good indicator for borrowers who are likely to default. <br> **You will see later in Bivariate analysis how this has influence on *delta_bins* variable as well**

In [ ]:
indicators = []
indicators.append("Pct_Payment_Received")

In [ ]:
indicators

In [ ]:
for col in df_numeric.columns:
    plt.figure(figsize=(10,6))
    sns.boxplot(x="annual_inc_bins", y=col,
                data=df)
    plt.show()

#### <font color="asparagus"> *Observations and Comments*:<font> 
$\Rightarrow$ <font color="asparagus"> Borrowers with high income borrow the higest amount. They also have made more inquiries in the past 6 months as compared to others. They have a lot more open accounts than borrowers with lower income than them. <br> $\Rightarrow$ Since they tend to borrow more, the total payment received is also more. <font><br>

In [ ]:
for col in df_numeric.columns:
    plt.figure(figsize=(10,6))
    sns.boxplot(x="funded_amnt_bins", y=col,
                data=df)
    plt.show()

#### <font color="asparagus"> *Observations and Comments*:<font> 
$\Rightarrow$ <font color="asparagus"> Higher the funded amount, higher is the interest rate as well.<br>$\Rightarrow$ When we look at the inquiries made in the last 6 months vs funded_amnt_bins, 30000-35000 categories have only one outlier. Borrowers in the range 35000-40000 have not made any inquiries.<br>$\Rightarrow$ People that borow a high amount, tend to have more open accounts too.<font><br>

In [ ]:
for col in df_numeric.columns:
    plt.figure(figsize=(10,6))
    sns.boxplot(x="int_rate_bins", y=col,
                data=df)
    plt.show()

$\Rightarrow$ <font color="asparagus"> There are quite a few columns that seem to be heavily skewed towards one value. Eg: total_rec_late_fee, pub_rec, delinq_2yrs, annual_inc<br>
$\Rightarrow$ <font color="asparagus"> An IQR analysis will help us get a sense of the number of outliers 

### <font color='skyblue'>  II.8.2. IQR Analysis<font>

In [ ]:
column_quantile_info ={}
for col in df_numeric.columns:
    Q1 = np.quantile(df[col], 0.25)
    Q2 = np.quantile(df[col], 0.5)
    Q3 = np.quantile(df[col], 0.75)
    Q4 = np.quantile(df[col], 0.99)
    Q5 = np.quantile(df[col], 0.01)
    IQR = Q3 - Q1
    column_quantile_info[col] = [Q1, Q2, Q3, IQR, Q3 + 1.5*IQR, Q1 - 1.5*IQR, Q4, Q5]

$\Rightarrow$ <font color="asparagus"> Let's just say outliers are those that are in the top 1% or bottom 1 %

In [ ]:
upper_outlier_in_every_column = {}

In [ ]:
for col in df_numeric.columns:
    #print((np.where(df[col] > column_quantile_info[col][6])))
    upper_outlier_in_every_column[col] = np.where(df[col] > column_quantile_info[col][6])

In [ ]:
for col in df_numeric.columns:
    print(len(upper_outlier_in_every_column[col][0]))

$\Rightarrow$ <font color="asparagus"> That's a lot of outliers in every column. Let's try to see if there are any common rows among all columns <font>

In [ ]:
common = np.intersect1d(upper_outlier_in_every_column["int_rate"], upper_outlier_in_every_column["last_pymnt_amnt"])

In [ ]:
for col in df_numeric.columns:
    if len(upper_outlier_in_every_column[col][0]) > 300:
        common = np.intersect1d(common, upper_outlier_in_every_column[col][0])
        print(col, ": ", common)

In [ ]:
common

In [ ]:
lower_outlier_in_every_column = {}

In [ ]:
for col in df_numeric.columns:
    #print((np.where(df[col] > column_quantile_info[col][6])))
    lower_outlier_in_every_column[col] = np.where(df[col] < column_quantile_info[col][7])

In [ ]:
for col in df_numeric.columns:
    print(len(lower_outlier_in_every_column[col][0]))

In [ ]:
lower_common = lower_outlier_in_every_column["funded_amnt"][0]

In [ ]:
for col in df_numeric.columns:
    if len(lower_outlier_in_every_column[col][0]) > 300:
        lower_common = np.intersect1d(lower_common, lower_outlier_in_every_column[col][0])
        print(col, ": ", lower_common)

In [ ]:
lower_common

$\Rightarrow$ <font color="asparagus"> There are no common rows among all these columns.<br> $\Rightarrow$ *Side note*: There are quite a few outliers, in general. For example, there is an entry of a borrower with 6 million dollars as the annual income. But I did not see any shortcomings in the analysis by leaving it in. <br> $\Rightarrow$ I eventually ended up leaving all the rows in, besides dropping so many unique rows(outliers of every column), the data would have shrunk drastically. <font>

# <font color='goldenrod'> III. Data Analysis </font>

### <font color='skyblue'>  III.1. Univariate analysis<font>

#### <font color="asparagus"> *Observations and Comments*:<font><br>
$\Rightarrow$ <font color="asparagus">Previously when we looked at box plots in *II.7.1*, we were mainly focused on the outliers. Let's look at them now to get some details about the variables.<br> $\Rightarrow$ We will also look at the distribution plots for some numeric columns too. <font>

#### <font color="asparagus"> *Segregating the columns into categorical and numerical explicitly* <font> 

In [ ]:
cat_cols = ["grade", "sub_grade", "term", "home_ownership", 
            "verification_status", "loan_status", 
            "purpose", "zip_code", "addr_state", "annual_inc_bins", 
            "funded_amnt_bins", "int_rate_bins", "emp_length_bins", 
            "delta", "delta_bins", "Pct_Payment_bins"]

In [ ]:
not_cat_cols = [col for col in df.columns if col not in cat_cols]

In [ ]:
df_numeric = df.select_dtypes(include=np.number)

In [ ]:
cont_cols = df_numeric.columns.to_list()

In [ ]:
cont_cols

### <font color='skyblue'>  III.1.1. Box Plots contd.<font>

In [ ]:
list(df.loan_status.unique())

In [ ]:
fig, axes = plt.subplots(5,5, figsize=(15,20))
plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
for col, ax in zip(cont_cols, axes.flatten()):
    sns.boxplot(y=col, x= "loan_status", data=df, orient='v', ax=ax)
    ax.tick_params(colors="black", which="both")
    ax.set_xticklabels(['Fully Paid', 'Charged Off', 'Current'], rotation=45,color="black")
    ax.set(xlabel=None)
plt.show()


#### <font color="asparagus"> *Observations and Comments*:<font><br>
#### I am restricting to variables that might give us some insight about the data. If it is not revealing a lot, I will skip it in the comments below.
    
$\Rightarrow$ <font color="asparagus"> funded_amnt: <font>
    
For the loans that are currently running, the median of the funded amount lies around 15000 dollars. Charged off and Fully Paid loans hover around 10,000 dollars.
    
$\Rightarrow$ <font color="asparagus"> term: <font>
    
Currently running loans are borrowers that took 5 year loans, whereas *Charged Off* and *Fully Paid* loans have both 3 and 5 year loans.

$\Rightarrow$ <font color="asparagus"> int_rate: <font>
    
Interest rate is high for currently running loans, *fully paid* loans have the lowest interest rates. This indicates that higher interest rate attracts more defaults.

$\Rightarrow$ <font color="asparagus"> emp_length: <font>
    
Employees with an experience of around 4 years seem to successfully pay off their loans, whereas the defaulters seem to have close to 5 years of experience. Currently running loans have more than 7 years of experience.

$\Rightarrow$ <font color="asparagus"> annual_inc: <font>
    
The ones with a lower annual income are more likely to defualt on their loans. When we look at the box plot of annual_inc vs *Fully Paid* loans, we see that it has borrowers with very less to really high incomes.

$\Rightarrow$ <font color="asparagus"> dti: <font>
    
dti is around the same for all the loan statuses.    
    
$\Rightarrow$ <font color="asparagus"> inq_last_6mnths: <font>
    
The borrowers who defaulted seem to enquire at least once in a space of 6 months. There are outliers though, so it is hard to conclude a lot from this.        

$\Rightarrow$ <font color="asparagus"> total_payment: <font>
    
As expected, the payment received is th least for the defaulters. Currently running loans seem to be of higher value than the ones that charged off. Again, there are outliers, but my comments are focused on median values across the board.
    

$\Rightarrow$ <font color="asparagus"> Percent Payment received: <font>
    
This column that we derived also indicates the same thing as *total_payment*. *Charged Off* loans are in general paying less. Currently running loans seem to be close to 100% paid for.     

$\Rightarrow$ <font color="asparagus"> Issue year: <font>
    
Most of the *fully paid* and *charged off* loans were issued some time in the year 2011. Currently running loans are close to 2012, late 2011. 
    
$\Rightarrow$ <font color="asparagus"> Delta: <font>
    
This is a column we derived earlier. To recall, it indicates the amount of time passed from issue date to last payment. Most of the *fully paid* loans' last payment was 3 years ago, with some outliers ~ 6 years, indicating that we might have data which includes loans given in the past 6 years. We can see *Charged Off* box plot suggesting that we might have data > 6 years as well(with the outliers). 

### <font color='skyblue'>  III.1.2. Probability Distribution <font>

#### <font color="asparagus"> *Observations and Comments*:<font><br>
From our box plot analysis, I think it makes sense looking at the probablity distribution of funded_amnt, annual income, interest rate, emp_length, percentage payment received just to get a glimpse of the pdf.

In [ ]:
distplots_cols = ["funded_amnt", "annual_inc", "int_rate", "Pct_Payment_Received", "emp_length"]

In [ ]:
plt.figure(figsize=(10,7))
for idx, col in enumerate(distplots_cols):
    ax = sns.displot(data=df, x=col, kde=True)
    plt.show()

#### <font color="asparagus"> *Observations and Comments*:<font><br>

$\Rightarrow$ <font color="asparagus"> annual_inc: <font>
    
Seems like most of the borrowers have incomes on the lower end. With the spike somwhere around 50000 to 100,000, calling describe() on annual_inc will give a us a clear picture.

$\Rightarrow$ <font color="asparagus"> int_rate: <font>
    
Most of the interest rate is in the range: 10-12.5%. With the highest spike at 7.5%

$\Rightarrow$ <font color="asparagus"> emp_length: <font>
    
Two spikes, one in the 0-2 years experience range and the other in the >10 years experience. 
<br> *Side note:* displots can also be plotted after z scoring for better detection of outliers. 


In [ ]:
df.annual_inc.describe()

# <font color='goldenrod'> III.2. Bivariate analysis </font>

#### <font color="asparagus"> *Comments*:<font> 
#### <font color="asparagus"> $\Rightarrow$ We've already studied the relation in between variables in the section *II.7. Correlation*.<br> $\Rightarrow$ We identified strongly correlated variables, removed some redundant columns.<br> $\Rightarrow$ We identified moderately and weakly correlated variables as well.<br> $\Rightarrow$ But now it's time to find some good indicators with groupbys, pivot tables and plots<font> 

### <font color='skyblue'>  III.2.1. Categorical columns with a keen focus on loan_status <font> 

<font color="asparagus"> Here are all the categorical columns. We won't group by all of them, but only ones that give us a good inference

In [ ]:
cat_cols

##### $\Rightarrow$ <font color="asparagus"> III.2.1.1. Grade <font>

In [ ]:
(df.groupby(by=["grade"]).mean())

In [ ]:
(df.groupby(by=["loan_status", "grade"]).mean())

#### <font color="asparagus"> *Observations and Comments*:<font><br>
$\Rightarrow$ <font color="asparagus">You can see that borrowers with bad credit ratings tend to borrow more money. Let's try some pivot tables and then plot a pie chart. <font>

In [ ]:
df.pivot_table(index="grade",columns=["loan_status"], values="funded_amnt", aggfunc="mean")

#### $\Rightarrow$ <font color="asparagus"> Let's get some numbers<font>
1. How many people of a certain grade borrowed?
2. How many of them defaulted, how many are currently paying/have fully paid?

#### $\Rightarrow$ <font color="asparagus">  Getting the grade-wise percentage for the three loan statuses we have

#### Let's look at the distribution of charged off loans based on grades



In [ ]:
df_chargedOff = df[df["loan_status"] == "Charged Off"].groupby(by="grade").size()

In [ ]:
df_chargedOff

In [ ]:
grade_labels = sorted(df.grade.unique())

In [ ]:
plt.figure(figsize=(10,10))
plt.pie(df_chargedOff,labels=grade_labels,autopct='%.0f%%', textprops={'color':"w"})
plt.show()

#### <font color="asparagus"> $\Rightarrow$ Observation and comments:
1. Looking at this, it seems B,C D are more likely to default. But it kind of goes against our assumption: "people with lower credit score are more likely to default". 
2. It seems to be deceptive. We must see gradewise percent of people that have defaulted. 

#### <font color="asparagus"> $\Rightarrow$  Gradewise distribution

#### <font color="asparagus"> $\Rightarrow$ Before we proceed further, let's define a generic function that returns the percentage of charged Off, current and fully paid borrowers.<br> $\Rightarrow$We must provide the bins across which we want to calucate the aforementioned percentags as an input argument. <br> $\Rightarrow$Finally, we use this to plot pie charts

In [ ]:
def calculate_groupwise_loan_status_percentage(bins_list, df, col):
    groupwise_dist ={}
    for curr_bin in bins_list:
        nChargedOff = len(df[(df[col] == curr_bin) & (df["loan_status"] == "Charged Off")])
        nCurrent = len(df[(df[col] == curr_bin) & (df["loan_status"] == "Current")])
        nFullyPaid = len(df[(df[col] == curr_bin) & (df["loan_status"] == "Fully Paid")])
        totalNoOfPeople = len(df[df[col] == curr_bin])
        if totalNoOfPeople > 0:
            groupwise_dist[curr_bin] = [round(100 *nChargedOff/totalNoOfPeople,2), round(100 *nCurrent/totalNoOfPeople,2), round(100 *nFullyPaid/totalNoOfPeople,2)]
        else:
            groupwise_dist[curr_bin] = [0] *3
    return groupwise_dist

#### Defining another function that will return a pivot table, indexed by a particular column. It also also three new columns: pct_chargedOff, pct_current, pct_fullyPaid and return the grouped df. <br> This function will regularly be consumed by other columns too.

In [ ]:
def get_grouped_df(df, col):
    df_grouped = df.groupby([col, "loan_status"]).loan_status.count().unstack().fillna(0)
    df_grouped["pct_chargedOff"] = 100 *(df_grouped["Charged Off"]) /( df_grouped["Charged Off"] + df_grouped["Current"] + df_grouped["Fully Paid"])
    df_grouped["pct_current"] = 100 *(df_grouped["Current"]) /( df_grouped["Charged Off"] + df_grouped["Current"] + df_grouped["Fully Paid"])
    df_grouped["pct_fullyPaid"] = 100 *(df_grouped["Fully Paid"]) /( df_grouped["Charged Off"] + df_grouped["Current"] + df_grouped["Fully Paid"])
    return df_grouped

In [ ]:
grade_wise_split = calculate_groupwise_loan_status_percentage(grade_labels, df, "grade")

In [ ]:
grade_wise_split

In [ ]:
#plt.figure(figsize=(10,15))
for label in grade_labels:
    plt.pie(grade_wise_split[label],labels=["Charged Off", "Current", "Fully Paid"], autopct='%.0f%%', textprops={'color':"w"})
    plt.title(f"Grade {label}")
    plt.show()


#### <font color="asparagus"> $\Rightarrow$ Observation and comments:
1. Now the above pie charts tell the true story and it is in line with our initial assumption: Bad credit score = More likely to default<br>
2. As you can see Grade A has only 6% charged off loans, and the lower you go the in grade, the higher the percentage. Eg: Grade G: Charged off percentage: 32%

In [ ]:
# I can safely append grade to the indicators list based on our analysis above
indicators.append("grade")

##### $\Rightarrow$ <font color="asparagus"> III.2.1.2. Home Ownership <font>

In [ ]:
(df.groupby(by=["home_ownership", "loan_status"]).mean())

#### <font color="asparagus"> *Observations and comments* <font>
$\Rightarrow$ The table above does not seem to give us a good picture. Can't seem to get a good segragation.<br> $\Rightarrow$ Let's define another function that will return a pivot table, indexed by a particular column. It also adds three new columns: pct_chargedOff, pct_current, pct_fullyPaid and return the grouped df. <br> $\Rightarrow$This function will regularly be consumed by other columns too.


In [ ]:
home_owner_group = get_grouped_df(df, "home_ownership")

In [ ]:
home_owner_group

#### <font color="asparagus"> *Observations and comments* <font>
$\Rightarrow$ The above table looks cleaner. But there doesn't seem to be any pattern emerging. I was hoping to see borrowers with mortgage and rent to have more defaults, but that's not the case. *OTHER* has the highest charged off percentage.<br> $\Rightarrow$  Home_ownership is probably not a good indicator for default

##### $\Rightarrow$ <font color="asparagus"> III.2.1.3. Verification Status

 #### <font color="asparagus" > <br> 1. Not verified: Default Percentage: 1266% <br> 2. Source Verified: 14.36% <br> 3. Verified: 16.01% <font>

In [ ]:
(df.groupby(by=["verification_status", "loan_status"]).mean())

In [ ]:
verif_status_group = get_grouped_df(df,"verification_status")

In [ ]:
verif_status_group

#### <font color="asparagus"> *Observations and comments* <font>
$\Rightarrow$ No pattern emerging here other. 

##### $\Rightarrow$ <font color="asparagus"> III.2.1.4. Purpose

In [ ]:
(df.groupby(by=["purpose"]).mean())

In [ ]:
purpose_grouped = get_grouped_df(df,"purpose")

In [ ]:
purpose_grouped

#### <font color="asparagus"> *Observations and comments* <font>
Certain purposes pop out. Let's try a pie chart.

In [ ]:
purpose_labels = sorted(df.purpose.unique())

In [ ]:
purpose_dist = calculate_groupwise_loan_status_percentage(purpose_labels, df, "purpose")

In [ ]:
# Plotting pie chart
for label in purpose_labels:
    plt.pie(purpose_dist[label],labels=["Charged Off", "Current", "Fully Paid"], autopct='%.0f%%', textprops={'color':"black"})
    plt.title(f" Purpose: {label}")
    plt.show()

#### <font color="asparagus"> *Observations and Comments*:<font> 
Small business borrowers seem to default a lot more. $\Rightarrow 26%.<br> The others are fairly even around 80 - 85%.<br> Loans borrowed for wedding seem to have the lowest default percentage
#### <font color="asparagus"> This can also be a good indicator to decide whether the loan is going to be defaulted or not.<font>

In [ ]:
# I can safely append purpose to the indicators list based on our analysis above
indicators.append("purpose")

#### <font color="asparagus"> $\Rightarrow$ A stacked bar plot says the same thing. Small businesses have more percent of charged off

In [ ]:
loan_labels = ["Charged Off", "Current", "Fully Paid"]
chargedOff = []
for label in purpose_labels:
    chargedOff.append(purpose_dist[label][0])
chargedOff = np.array(chargedOff) 

current = []
for label in purpose_labels:
    current.append(purpose_dist[label][1])
current = np.array(current) 

Fully = []
for label in purpose_labels:
    Fully.append(purpose_dist[label][2])
Fully = np.array(Fully)

In [ ]:
# plot stacked bar
ax = plt.figure(figsize=(10,10))
plt.bar(purpose_labels, Fully, color='mediumseagreen')
plt.bar(purpose_labels, current, bottom=Fully, color='tab:olive')
plt.bar(purpose_labels, chargedOff, bottom=Fully+current, color=(0.5,0.3,0.3))
plt.xticks(rotation="vertical")
plt.yticks(color=(0.7,0.9,0.9))
plt.show()

##### $\Rightarrow$ <font color="asparagus"> III.2.1.5. Employee Experience

In [ ]:
df.pivot_table(index=["emp_length_bins"], columns=["loan_status"], values="funded_amnt", aggfunc="mean")

#### <font color="asparagus"> $\Rightarrow$ The more experienced borrowers tend to borrow more money <font>

In [ ]:
emp_exp_list = sorted(df.emp_length_bins.unique().to_list())

In [ ]:
emp_exp_wise_dist = calculate_groupwise_loan_status_percentage(emp_exp_list, df, "emp_length_bins")

In [ ]:
emp_exp_wise_dist

In [ ]:
# Plotting pie chart
for label in emp_exp_list:
    plt.pie(emp_exp_wise_dist[label],labels=["Charged Off", "Current", "Fully Paid"], autopct='%.0f%%', textprops={'color':"w"})
    plt.title(f"Emp experience: {label}")
    plt.show()


#### <font color="asparagus"> Shouldn't people with less experience default more often? But I do not observe any pattern here.<font> <br>

#### <font color="asparagus"> The following cells will help give us better insight

In [ ]:
emp_exp_group = get_grouped_df(df,"emp_length_bins")

In [ ]:
emp_exp_group

In [ ]:
emp_exp_group.sort_values(["pct_chargedOff"], ascending=False)

#### <font color="asparagus"> *Observations and Comments*<br>$\Rightarrow$ I see people with more than 10 years of experience, defaulting more often than people with less experience.<br> $\Rightarrow$ But the increase in percentage is very small. For the most part the percentage is ~ 15%<br> $\Rightarrow$ Can't really put emp_length as a very strong indicator of default.<br> $\Rightarrow$<font color="asparagus"> *Side Note*: Borrowers with 8-10 experience have the lowest percent of default <font>

##### $\Rightarrow$ <font color="asparagus"> III.2.1.6. Annual Income

In [ ]:
annual_inc_labels = sorted(df.annual_inc_bins.unique())

In [ ]:
annual_inc_wise_dist = calculate_groupwise_loan_status_percentage(annual_inc_labels, df, "annual_inc_bins")

In [ ]:
# Plotting pie chart
for label in annual_inc_labels:
    plt.pie(annual_inc_wise_dist[label],labels=["Charged Off", "Current", "Fully Paid"], autopct='%.0f%%', textprops={'color':"w"})
    plt.title(f"Annual inc(thousands): {label}")
    plt.show()

#### <font color="asparagus"> *Observations and Comments*:<font> 
The percentage of defaulting borrowers with annual income in between $0-$20k is double that of those make more than 100k dollars. We can further split >100k and potentially observe the same pattern

In [ ]:
# I can safely append annual inc to the indicators list based on our analysis above
indicators.append("annual_inc")

In [ ]:
indicators

##### $\Rightarrow$ <font color="asparagus"> III.2.1.7. Funded Amount

In [ ]:
funded_amnt_labels = sorted(df.funded_amnt_bins.unique())

In [ ]:
funded_amnt_labels

In [ ]:
funded_amnt_dist = calculate_groupwise_loan_status_percentage(funded_amnt_labels, df, "funded_amnt_bins")

In [ ]:
funded_amnt_dist

In [ ]:
# Plotting pie chart
for label in funded_amnt_labels:
    plt.pie(funded_amnt_dist[label],labels=["Charged Off", "Current", "Fully Paid"], autopct='%.0f%%', textprops={'color':"w"})
    plt.title(f" funded_amount(1000s): {label}")
    plt.show()

#### <font color="asparagus"> $\Rightarrow$ Observations:
1. Higher the amount, more is the charged off percentage. Eg: In the range 30000-35000: the percentage of borrowers who charged off is 22%
2. Charged off percentage for 0-5000 is 14%.
3. Adding this to the list of indicators

In [ ]:
indicators.append("funded_amnt")

In [ ]:
indicators

##### $\Rightarrow$ <font color="asparagus"> III.2.1.8. Rate of Interest

In [ ]:
int_rate_labels = (df.int_rate_bins.unique())

In [ ]:
int_rate_dist = calculate_groupwise_loan_status_percentage(int_rate_labels, df, "int_rate_bins")

In [ ]:
# Plotting pie chart
for label in int_rate_labels:
    plt.pie(int_rate_dist[label],labels=["Charged Off", "Current", "Fully Paid"], autopct='%.0f%%', textprops={'color':"w"})
    plt.title(f" int rate: {label}%")
    plt.show()

#### <font color="asparagus"> *Observations and Comments*:<font> 
$\Rightarrow$ We can conclude that higher interest rates, attract more defaults. Eg: For int_rate in the range 0-8%, % charged off = 5%<br>$\Rightarrow$ int_rate >16% has % charged Off = 27% <font color="asparagus"> **A huge increase**<font>

In [ ]:
# I can safely append int_rate to the indicators list based on our analysis above
indicators.append("int_rate")

In [ ]:
indicators

##### $\Rightarrow$ <font color="asparagus"> III.2.1.9. Public records

In [ ]:
df.pub_rec.value_counts()

In [ ]:
pub_rec_group = get_grouped_df(df,"pub_rec")

In [ ]:
pub_rec_group

In [ ]:
pub_rec_group.sort_values("pct_chargedOff", ascending=False)

#### <font color="asparagus"> pub_rec according to the data dictionary:  Number of derogatory public records. <font> 
$\Rightarrow$ We might think that more the derogatory records, higher the defaults. But that's not what we see in this data.<br> $\Rightarrow$ Borrowers with 3 and 4 pub_rec have fully paid, whereas borrowers with only 1 derogatory record have defaulted more often than those with 2 derogatory records
Therefore, I cannot add this as to the list of indicators

##### <font color="asparagus"> $\Rightarrow$ III.2.1.10. Number of inquiries in the last 6 months

In [ ]:
inq_6mnths_group = get_grouped_df(df, "inq_last_6mths")

In [ ]:
inq_6mnths_group

In [ ]:
inq_6mnths_group.sort_values(["pct_chargedOff"], ascending=False)

#### <font color="asparagus"> $\Rightarrow$ Now this is a good indicator of default. Borrowers who have inquired more in the past 6 months are more likely to default

In [ ]:
indicators.append("inq_last_6mnths")

#### <font color="asparagus"> $\Rightarrow$ III.2.1.11. State Address

In [ ]:
addr_state_group = get_grouped_df(df, "addr_state")

In [ ]:
addr_state_group.sort_values("pct_chargedOff", ascending=False)

In [ ]:
plt.figure(figsize=(10,15))
sns.barplot(x=addr_state_group.index, y=addr_state_group["pct_chargedOff"])
plt.xticks(rotation="vertical")
# plt.yticks(color="w")
plt.show()
plt.tight_layout()

#### <font color="asparagus"> Borrowers from the state of Nebraska(NE) are more likely to default than others. $\Rightarrow$ 60% <font> <br> We can add addr_state to the list of indicators as well

In [ ]:
indicators.append("addr_state")

In [ ]:
indicators

#### <font color="asparagus"> $\Rightarrow$ III.2.1.12. Loan Term

In [ ]:
term_grouped = get_grouped_df(df,"term")

In [ ]:
term_grouped.sort_values("pct_chargedOff", ascending=False)

#### <font color="asparagus"> Borrowing money for a term period of 5 years are twice as likely to default than those whose term period is 3 years.(22.6% vs 11.09%). <font>
We need to understand why though. What's so specific about the 5 year plan, that makes borrowers default?<br> Let's do a bivariate analysis of term period with other variables in our indicators' list.

In [ ]:
sns.boxplot(y="Pct_Payment_Received", x="term", data=df)

#### <font color="asparagus"> *Observation and comments*<font>
It makes sense that the amount of payment received is less for the 5 year loan since they default more often

We know that the worse your credit rating(grade), the more likely it is that you will default. But I can't see much of a pattern here

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(y="grade", x="term", data=df)
#plt.xticks(color="w")
#plt.yticks(color="w")

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(y="annual_inc_bins", x="term", data=df)
#plt.xticks(color="w")
#plt.yticks(color="w")

#### <font color="asparagus"> *Observation and comments*<font>
From our previous analysis, lower the income the more likely it is that the borrower will default. 
The above plot shows just that. Borrowers with income in the range \$0-\$40,000 opt for the 5 year loan, which explains the increase of defaults in the 5 year plan

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(y="purpose", x="term", data=df)
#plt.xticks(color="w")
#plt.yticks(color="w")
plt.show()

#### <font color="asparagus"> *Observation and comments*<font>
We know loans for the purpose of small businesses tend to default more, but the boxplot says that they go for both 3 and 5 year terms. May be a percentage will help?

In [ ]:
chargedOffSum = len(df[(df.purpose == "small_business") & (df["loan_status"] =="Charged Off")])
n5yrSum = len(df[(df.purpose == "small_business") & (df["loan_status"] =="Charged Off") & (df.term == 5)])
n3yrSum = len(df[(df.purpose == "small_business") & (df["loan_status"] =="Charged Off") & (df.term == 3)])


In [ ]:
100* n5yrSum/chargedOffSum, 100* n3yrSum/chargedOffSum


#### <font color="asparagus"> *Observation and comments*<font>
No, it seems small business borrowers have been given 3 year term

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(y="funded_amnt_bins", x="term", data=df)
#plt.xticks(color="w")
#plt.yticks(color="w")
plt.show()

#### <font color="asparagus"> *Observation and comments*<font>
From before, higher the funded amount, higher the charged off percentage. But it seems I cannot get any pattern here.<br> But we also know that higher the funded amount, higher is the interest given to them. <br>So there might be something when we do a boxplot of term with interest rate

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(y="int_rate", x="term", data=df)
#plt.xticks(color="w")
#plt.yticks(color="w")
plt.show()

#### <font color="asparagus"> *Observation and comments*<font>
Bingo! As suspected, the interest rate is high for 5 year term! And we know that higher interest rates attract more defaults. Therefore, this is another contributing factor to the higher number of defaults in the 5 year loans(double that of 3 year loans)

In [ ]:
plt.figure(figsize=(10,10))
sns.boxplot(y="inq_last_6mths", x="term", data=df)
#plt.xticks(color="w")
#plt.yticks(color="w")
plt.show()

#### <font color="asparagus"> *Observation and comments*<font>
Median of inq_last_6mths is 1 for the 5 year loan. But there are outliers in the 3 year loan as well. Can't really conclude much. This is a contributing factor but not very strong, I guess.

In [ ]:
plt.figure(figsize=(10,50))
sns.boxplot(y="addr_state", x="term", data=df)
#plt.xticks(color="w")
#plt.yticks(color="w")
plt.show()

#### <font color="asparagus"> *Observation and comments*<font>
Can't conclude much here. The term is all over the place. I mainly wanted to see what's going on in NE, NA, AK, SD since they have the highest defaults. In fact, NE, the state with the highest defaults has purely 3 year loans

##### <font color="asparagus"> $\Rightarrow$ III.2.1.13. Delta_bins

In [ ]:
delta_bins_group = get_grouped_df(df,"delta_bins")

In [ ]:
delta_bins_group

#### <font color="asparagus"> *Observations and Comments* <font>
The LC club has two term plans: 3 and 5 years. The delta_bins are indicating the time passed from the issue date. <br>Directly analyzing the table won't make sense, because obviously the ones where more than 5 years have passed are the borrowers who have defaulted. <br> We should rather add another level of segregation. <br> We will choose the percentage of payment received for this purpose. $\Rightarrow$ We know that defaulters will have payment percentage below 100. So let's focus on that.

#### This is the list of all the borrowers who borrowed money within the past year, whose payments are not on time and have less than a 100% 

In [ ]:
df_borrowed_within_a_yr = df[(df.Pct_Payment_Received < 100) & (df.delta_bins =="0-1")]

In [ ]:
df_borrowed_within_a_yr

#### So now let's check the loan_status of these guys

In [ ]:
df_borrowed_within_a_yr.loan_status.value_counts()

#### Bingo! All of them end up defaulting on their loan

#### So is the case for the other bins.

In [ ]:
df[(df.Pct_Payment_Received < 100) & (df.delta_bins =="1-2")]["loan_status"].value_counts()

In [ ]:
df[(df.Pct_Payment_Received < 100) & (df.delta_bins =="2-3")]["loan_status"].value_counts()

In [ ]:
df[(df.Pct_Payment_Received < 100) & (df.delta_bins =="3-4")]["loan_status"].value_counts()

#### In the 4-5 bin, there are still people whose loan status is current. 

In [ ]:
df[(df.Pct_Payment_Received < 100) & (df.delta_bins =="4-5")]["loan_status"].value_counts()

#### $\Rightarrow$  But they are more likely to default in the future, in my opinion. It probably depends on the percentage of payment received. <br> $\Rightarrow$ Because by this time(4-5 year from loan issue date), this percent of amount payed should be very high, close to 100%, so let's check as well

In [ ]:
df[df.delta_bins =="4-5"]["Pct_Payment_Received"].value_counts()

#### <font color="asparagus"> Let's split percent payment received in to two categories: less than 80% and greater than 80%

In [ ]:
Pct_Payment_categories_list = [0, 80, 100]
Pct_Payment_labels_list = ["0-80",">80"]
df["Pct_Payment_bins"] = pd.cut(df["Pct_Payment_Received"], bins=Pct_Payment_categories_list,
                              labels=Pct_Payment_labels_list)

In [ ]:
df_payment_delta_bin_group = df.groupby(["delta_bins", "Pct_Payment_bins"]).Pct_Payment_bins.count().fillna(0).unstack()

In [ ]:
df_payment_delta_bin_group

#### <font color="asparagus"> *Observations and Comments*:<font><br>
$\Rightarrow$ <font color="asparagus">You can see that 4-5 bin has 71 borrowers who have payed less than 80% of their loan. Therefore, we can see pct_payment and delta_bin together are a great indicators of loan defaults

#### <font color="asparagus">Since the bank only has two terms, it is best to check at the term boundaries. i.e. For the 3 year loan, check bin = 2-3 and percent_payment_bin = 0-80. 

In [ ]:
df[(df.Pct_Payment_bins == "0-80") & (df.term==3) & (df.delta_bins=="2-3")]["loan_status"].value_counts()

#### <font color="asparagus"> As you can see all, of them default

#### <font color="asparagus"> Similarly for the 5 year loan...

In [ ]:
df[(df.Pct_Payment_bins == "0-80") & (df.term==5) & (df.delta_bins=="4-5")]["loan_status"].value_counts()

#### <font color="asparagus"> Most of them default too! : )

In [ ]:
indicators.append("delta_bins") # adding this to the list of indicators. delta_bins can be used along with
#pct_payment_received to gain insight on loan defaults

### <font color='skyblue'>  III.2.2. In between rest of the columns(minus loan_status) <font> 

#### <font color="asparagus"> *Comments:* <font>
Let's restrict ourselvs to the columns in the indicators, to see if there are any patterns.<br>$\Rightarrow$ grade vs int_rate<br>
$\Rightarrow$ purpose vs int_rate<br>
$\Rightarrow$ addr_state vs int_rate
$\Rightarrow$ fended_amnt vs int_rate    


In [ ]:
indicators

#### $\Rightarrow$ <font color='asparagus'>grade vs int_rate

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y="grade", x="int_rate", data=df)

#### <font color="asparagus"> *Observations and Comments:* <font>
Interest rate is really high for Grade G borrowers. Explains why they default a lot more.<br>We already know that high interest rates attract high defaults... Wonder what the process is to decide the interest rate for a particular borrower.

#### $\Rightarrow$ <font color='asparagus'>purpose vs int_rate

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y="purpose", x="int_rate", data=df)

#### <font color="asparagus"> *Observations and Comments:* <font>
Small businesses end up defaulting more. We can see that the interest is also on the higher side for borrowers borrowing money for the purpose of small_business 

#### $\Rightarrow$ <font color='asparagus'> addr_state vs int_rate

In [ ]:
plt.figure(figsize=(30,30))
sns.boxplot(y="addr_state", x="int_rate", data=df)

#### <font color="asparagus"> *Observations and Comments:* <font>
It's a bit all over the place. Can't really conclude much

#### $\Rightarrow$ <font color='asparagus'> funded_amnt vs int_rate

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y="funded_amnt_bins", x="int_rate", data=df)
plt.show()

#### <font color="asparagus"> *Observations and Comments:* <font>
We know from our previous analysis, higher the funded amount, higher is the interest rate and therefore the defaults. The above box plot explains the same thing.

#### $\Rightarrow$ Now let's look at some other factors. When we want a loan, there are certain questions to be asked by both the LC and the borrower:
$\Rightarrow$ What is the loan amount? <br> 
$\Rightarrow$ What is the rate of interest.<br> 
$\Rightarrow$ How many years we intend to repay the loan in?

#### <font color="asparagus"> $\Rightarrow$ int_rate vs term <br> 

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y="int_rate", x="term", data=df)
plt.show()

#### <font color="asparagus"> *Observations and Comments:* <font>
With the above plot, we can say that for 5 year loans, the LC has decided that the rate of interest should be around 15% and for the 3 year loan, it is around 11%

#### <font color="asparagus"> $\Rightarrow$ int_rate vs funded_amnt <br> 

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y="int_rate", x="funded_amnt_bins", data=df)
plt.show()

#### <font color="asparagus"> *Observations and Comments:* <font>
With the above plot, we can say that higher the funded_amnt(loan_amnt), higher is the interest rate

#### <font color="asparagus"> $\Rightarrow$ funded_amnt vs term <br> <font>

In [ ]:
plt.figure(figsize=(15,10))
sns.boxplot(y="funded_amnt", x="term", data=df)
plt.show()

#### <font color="asparagus"> *Observations and Comments:* <font>
With the above plot, we can say that larger funded amounts go with 5 year loan repayment plan. But there are so many outliers in the 3 year boxplot! Can't really say for sure : )

# <font color='goldenrod'> IV. Conclusion </font>

In [ ]:
indicators

<font color="asparagus"> 1. Percent of payment received and *delta_bins*(amount of time elapsed from loan issue date to the last payment date): <font>

    The LC lends money with two term plans: 3 and 5 year plans. Checking the percentage of the amount payed at the edge i.e. 2-3 or 4-5 will give us a good indication of whether the loan is going to be defaulted or not.
    Side note: This information, however is not available at the time of loan application as earlier pointed out. But if just look at this from a data analysis point of view, it does give good insights to predicting whether a currently running loan is going to be defaulted or not.  
<font color="asparagus"> 2. Grade: <font>
    
    If we say grade A > B > C > D > E > F > G, then lower the loan grade, higher is the chance of defaulting.
<font color="asparagus"> 3. Annual income: <font> 
    
    Borrowers with lower income have a higher chance of defaulting on the loan.
<font color="asparagus">4. Funded amount: <font> 
    
    Higher the funded amount, higher is the chance of defaulting.
<font color="asparagus"> 5. Interest rate: <font>
    
    High interest rates attrack higher percentage of defaults.
<font color="asparagus"> 6. Inquiries in the last 6 months: <font>
    
    More the number of inquiries made the borrower in the last 6 months, more likely it is for the borrower to default.
<font color="asparagus"> 7. State address: <font>
    
    Certain states have higher chance of defaulting. Eg: Nebraska

# <font color='goldenrod'> V. Recommendations </font>

1. Lower grade loans should be avoided as they have a higher chance of defaulting. 
2. We should try to lend money to people with a higher annual income and avoid ones on the lower side like 0 - 20,000 dollars.
3. If the loan amount asked is on the higher side, there is a higher chance of defaults. Funded amounts in the range of 0 to 15,000 dollars have around the same chances of defaults. Loan amounts greater 15,000 dollars have a higher chance of defaulting. Therefore, it's best to stay within 0 - 15,000 dollars. May within 20,000, if one is willing to take a little more risk. But safe option is within 15,000 dollars.
4. Try to look out loans with a lower interest rate, since they are less likely to be defaulted. Interest rates within 10% are the safest options. Anything beyond 14 % should definitely be avoided, within loans > 16% having the highest chances of defaulting.
5. Borrowers making more number of inquiries in the last 6 months are more likely to default. Borrowers who don't make any inquiries are the safes option. And borrowers making inquiries in the range 1-2 default more. Anything above that should definitely be avoided as the default rate >= 20%
6. Borrowers from certain states tend to default more. Nebraska should definitely be avoided as 60% of the loans have been defaulted. Nevada(NV) is far second with around 21.73% chance of defaulting. So a table can be shared to lenders to make an informed decision.